# COlumns defenition

In [1]:
CPU_SYSTEM_COL = "cpu_seconds_system"
MEMORY_SYSTEM_COL = "memory_mb_relative_system"
DISK_READ_BYTES_SYSTEM_COL = "disk_read_kb_system"
DISK_READ_COUNT_SYSTEM_COL = "disk_read_count_system"
DISK_WRITE_BYTES_SYSTEM_COL = "disk_write_kb_system"
DISK_WRITE_COUNT_SYSTEM_COL = "disk_write_count_system"
DISK_READ_TIME = "disk_read_time_ms_system"
DISK_WRITE_TIME = "disk_write_time_ms_system"
NETWORK_BYTES_SENT_SYSTEM_COL = "network_kb_sent_system"
NETWORK_PACKETS_SENT_SYSTEM_COL = "network_packets_sent_system"
NETWORK_BYTES_RECEIVED_SYSTEM_COL = "network_kb_received_system"
NETWORK_PACKETS_RECEIVED_SYSTEM_COL = "network_packets_received_system"

ENERGY_USAGE_SYSTEM_COL = "system_energy_usage_mwh"
DURATION_COL = "duration"
PC_TYPE = "PC_type"
PC_MANUFACTURER = "PC_manufacturer"
SYSTEM_FAMILY = "system_family"
PC_MODEL = "model"
MACHINE_TYPE = "machine_type"
DEVICE_NAME = "device_name"

## operating system info
OPERATING_SYSTEM = "operating_system"
OPERATING_SYSTEM_RELEASE = "operating_system_release"
OPERATING_SYSTEM_VERSION = "operating_system_version"

## cpu info
PROCESSOR_NAME = "processor_name"
PROCESSOR_PHYSICAL_CORES = "processor_physical_cores"
PROCESSOR_TOTAL_CORES = "processor_total_cores"
PROCESSOR_MAX_FREQ = "processor_max_frequency"
PROCESSOR_MIN_FREQ = "processor_min_frequency"

## RAM info
TOTAL_RAM = "total_ram"
PHYSICAL_DISK_NAME = "physical_disk_name"
PHYSICAL_DISK_MANUFACTURER = "physical_disk_manufacturer"
PHYSICAL_DISK_MODEL = "physical_disk_model"
PHYSICAL_DISK_MEDIA_TYPE = "physical_disk_media_type"
PHYSICAL_DISK_FIRMWARE_VERSION = "disk_firmware_version"
LOGICAL_DISK_NAME = "logical_disk_name"
LOGICAL_DISK_MANUFACTURER = "logical_disk_manufacturer"
LOGICAL_DISK_MODEL = "logical_disk_model"
LOGICAL_DISK_DISK_TYPE = "logical_disk_disk_type"
LOGICAL_DISK_PARTITION_STYLE = "logical_disk_partition_style"
LOGICAL_DISK_NUMBER_OF_PARTITIONS = "logical_disk_number_of_partitions"
PHYSICAL_SECTOR_SIZE = "physical_sector_size"
LOGICAL_SECTOR_SIZE = "logical_sector_size"
BUS_TYPE = "bus_type"
FILESYSTEM = "file_system"
BATTERY_DESIGN_CAPACITY = "design_battery_capacity"
FULLY_CHARGED_BATTERY_CAPACITY = "fully_charged_battery_capacity"

ENERGY_USAGE_PROCESS_COL = "energy_consumption_process_mWh"
CPU_PROCESS_COL = "cpu_usage_seconds_process"
MEMORY_PROCESS_COL = "memory_mb_relative_process"
DISK_READ_BYTES_PROCESS_COL = "disk_read_kb_process"
DISK_READ_COUNT_PROCESS_COL = "disk_read_count_process"
DISK_WRITE_BYTES_PROCESS_COL = "disk_write_kb_process"
DISK_WRITE_COUNT_PROCESS_COL = "disk_write_count_process"
PAGE_FAULT_PROCESS_COL = "number_of_page_faults_process"
NETWORK_BYTES_SENT_PROCESS_COL = "network_kb_sent_process"
NETWORK_PACKETS_SENT_PROCESS_COL = "network_packets_sent_process"
NETWORK_BYTES_RECEIVED_PROCESS_COL = "network_kb_received_process"
NETWORK_PACKETS_RECEIVED_PROCESS_COL = "network_packets_received_process"
PROCESS_ID_COL = "pid"

In [86]:
NON_NEGATIVE_COLS = [
    DURATION_COL,
    DISK_READ_BYTES_SYSTEM_COL,
    DISK_WRITE_BYTES_SYSTEM_COL,
    CPU_SYSTEM_COL,
    ENERGY_USAGE_SYSTEM_COL
]

NON_NEGATIVE_COLS2 = [
    DURATION_COL,
    DISK_READ_BYTES_SYSTEM_COL,
    DISK_WRITE_BYTES_SYSTEM_COL,
    CPU_SYSTEM_COL
]

OUTLIER_COLS = [
    DURATION_COL,
    MEMORY_SYSTEM_COL,
    DISK_READ_BYTES_SYSTEM_COL,
    DISK_WRITE_BYTES_SYSTEM_COL,
    CPU_SYSTEM_COL
]

PROCESS_ONLY_COLUMNS = [CPU_PROCESS_COL,
                        MEMORY_PROCESS_COL,
                        DISK_READ_BYTES_PROCESS_COL,
                        DISK_READ_COUNT_PROCESS_COL,
                        DISK_WRITE_BYTES_PROCESS_COL,
                        DISK_WRITE_COUNT_PROCESS_COL,
                        NETWORK_BYTES_SENT_PROCESS_COL,
                        NETWORK_PACKETS_SENT_PROCESS_COL,
                        NETWORK_BYTES_RECEIVED_PROCESS_COL,
                        NETWORK_PACKETS_RECEIVED_PROCESS_COL,
                        PAGE_FAULT_PROCESS_COL]

SYSTEM_ONLY_COLUMNS = [
    CPU_SYSTEM_COL,
    MEMORY_SYSTEM_COL,
    DISK_READ_BYTES_SYSTEM_COL,
    DISK_READ_COUNT_SYSTEM_COL,
    DISK_WRITE_BYTES_SYSTEM_COL,
    DISK_WRITE_COUNT_SYSTEM_COL,
    DISK_READ_TIME, DISK_WRITE_TIME,
    NETWORK_BYTES_SENT_SYSTEM_COL,
    NETWORK_PACKETS_SENT_SYSTEM_COL,
    NETWORK_BYTES_RECEIVED_SYSTEM_COL,
    NETWORK_PACKETS_RECEIVED_SYSTEM_COL,
    DURATION_COL,
    ENERGY_USAGE_SYSTEM_COL]

# Functions

In [3]:
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd
from measurements_model_pipeline.utils import calculate_and_print_scores
from sklearn.preprocessing import StandardScaler


def print_predictions(x_test, y_test, y_pred):
    for i, (actual, predicted) in enumerate(zip(y_test, y_pred)):
        print("Actual:", actual)
        print("Predicted:", predicted)
        if predicted > 100:
            print("Found bad prediction at row ", i)
            print(f"Row {i}: {x_test.iloc[i].to_dict()}")


def debug_predictions(X_test: pd.DataFrame, y_test: pd.Series, y_pred: pd.Series, top_n: int = 10):
    """
    Prints key statistics and highlights rows with the largest prediction errors
    to easily detect corrupted feature rows or prediction issues.
    """
    import pandas as pd
    import numpy as np

    # Ensure all inputs are properly formatted
    X_test = X_test.copy()
    y_test = pd.Series(y_test).reset_index(drop=True)
    y_pred = pd.Series(y_pred).reset_index(drop=True)

    # 1. General stats
    print("=== GENERAL STATS ===")
    print("X_test shape:", X_test.shape)
    print("y_test shape:", y_test.shape)
    print("y_pred shape:", y_pred.shape)
    print("\ny_test.describe():\n", y_test.describe())
    print("\ny_pred.describe():\n", y_pred.describe())

    # 2. Compute errors
    errors = (y_test - y_pred).abs()
    print("\nMax absolute error:", errors.max())
    print("Mean absolute error:", errors.mean())
    print("Median absolute error:", errors.median())

    # 3. Show top N largest errors
    top_errors_idx = errors.sort_values(ascending=False).head(top_n).index
    print(f"\n=== TOP {top_n} LARGEST ERRORS ===")
    for idx in top_errors_idx:
        print(f"Row {idx}:")
        print("  Features:", X_test.iloc[idx].to_dict())
        print("  Actual:", y_test.iloc[idx])
        print("  Predicted:", y_pred.iloc[idx])
        print("  Abs Error:", errors.iloc[idx])
        print("---")

    # 4. Optionally detect extreme feature values
    extreme_values = X_test[(X_test.abs() > 1e5).any(axis=1)]
    if not extreme_values.empty:
        print("\n!!! WARNING: Some feature rows have extremely large values (>1e5) !!!")
        print(extreme_values)
    else:
        print("\nNo extreme feature values detected.")


def run_pipeline(x_train, x_test, y_train, y_test):
    y_train = pd.Series(y_train.squeeze()).reset_index(drop=True)
    y_test = pd.Series(y_test.squeeze()).reset_index(drop=True)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(x_train)
    X_test_scaled = scaler.transform(x_test)

    X_train_scaled = pd.DataFrame(
        X_train_scaled,
        columns=x_train.columns,
        index=x_train.index
    )

    X_test_scaled = pd.DataFrame(
        X_test_scaled,
        columns=x_test.columns,
        index=x_test.index
    )

    model = HistGradientBoostingRegressor(max_depth=11, min_samples_leaf=10)
    model.fit(X_train_scaled, y_train)

    y_pred = pd.Series(model.predict(X_test_scaled)).reset_index(drop=True)

    print_predictions(X_test_scaled, y_test, y_pred)
    debug_predictions(X_test_scaled, y_test, y_pred, top_n=10)

    calculate_and_print_scores(y_test, y_pred)
    return scaler, model

In [4]:
import numpy as np


def process_df(df: pd.DataFrame, non_negative_cols: list[str], outlier_cols: list[str]) -> pd.DataFrame:
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    if non_negative_cols is not None:
        for col in non_negative_cols:
            df = df[df[col] >= 0]

    if outlier_cols is not None:
        for col in outlier_cols:
            low = df[col].quantile(0.01)
            high = df[col].quantile(0.99)
            df = df[(df[col] >= low) & (df[col] <= high)]

    return df

In [5]:
from sklearn.model_selection import train_test_split


def execute_model(x, y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    X_train = X_train.copy()
    X_test = X_test.copy()
    y_train = y_train.copy()
    y_test = y_test.copy()
    model = run_pipeline(X_train, X_test, y_train, y_test)
    return model

# Read system only data

In [6]:
system_df = pd.read_csv("system_df.csv", index_col=0)
system_df = process_df(system_df, NON_NEGATIVE_COLS, OUTLIER_COLS)
X = system_df.drop(columns=[ENERGY_USAGE_SYSTEM_COL])
y = system_df[ENERGY_USAGE_SYSTEM_COL]

In [7]:
system_scaler, system_model = execute_model(X, y)

Actual: 46.79920792
Predicted: 46.93861710120535
Actual: 9.73175896
Predicted: 11.090239005450739
Actual: 6.639591149999999
Predicted: 6.803750549435668
Actual: 10.06432029
Predicted: 10.856837072743357
Actual: 46.57991022
Predicted: 47.47016255798545
Actual: 58.852538
Predicted: 57.08574257045556
Actual: 58.92894890666667
Predicted: 58.204271360717854
Actual: 58.03732138
Predicted: 58.16061071354229
Actual: 17.359863913333335
Predicted: 13.293091677866617
Actual: 58.60084044
Predicted: 58.00133176793917
Actual: 5.104683386666667
Predicted: 13.052083520051864
Actual: 15.107442386666667
Predicted: 12.484069152471985
Actual: 57.90747493999999
Predicted: 57.511578311084016
Actual: 6.80002704
Predicted: 7.013047994528434
Actual: 57.85710318
Predicted: 57.810008742055615
Actual: 10.422580773333332
Predicted: 10.676607980856227
Actual: 10.74159607
Predicted: 11.597850145777091
Actual: 17.383713123333333
Predicted: 13.343071392038386
Actual: 12.790619799999998
Predicted: 11.476956026095761
Ac

# Collect data

In [70]:
from measurements_model_pipeline.dataset_creation.dataset_creator import DatasetCreator

dataset_creator = DatasetCreator()

KivyMD: Version 1.2.0 is deprecated and is no longer supported. Use KivyMD version 2.0.0 from the master branch (pip install https://github.com/kivymd/KivyMD/archive/master.zip)
Base: No event listeners have been created
Base: Application will leave


In [71]:
full_dataset = dataset_creator.create_dataset()

In [72]:
full_dataset.to_csv("process_system_full_df.csv", index=False)

# Read Process Data

In [119]:
process_system_df = pd.read_csv("process_system_full_df.csv")

## Get System - Process df

get a dataframe with the substruction of the total system and the specific process.

In [120]:
process_system_df.shape

(8183, 57)

In [121]:
process_system_df.head()

,duration,cpu_usage_seconds_process,memory_mb_relative_process,disk_read_kb_process,disk_read_count_process,disk_write_kb_process,disk_write_count_process,number_of_page_faults_process,network_kb_sent_process,network_packets_sent_process,...,logical_disk_partition_style,logical_disk_number_of_partitions,physical_sector_size,logical_sector_size,bus_type,file_system,design_battery_capacity,fully_charged_battery_capacity,energy_consumption_process_mWh,system_energy_usage_mwh
0,5.260686,0.0,0.574,0.0,0,0.0,0,147,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,2.665414,10.924691
1,5.318386,0.0,-0.238,0.0,0,0.0,0,1160,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,2.694649,11.044515
2,5.249743,0.0,0.258,0.0,0,0.0,0,66,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,2.659870,10.901966
3,5.307450,0.0,0.250,0.0,0,0.0,0,64,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,2.689108,11.021805
4,5.242309,0.0,0.199,0.0,0,0.0,0,484,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,2.656103,10.886528


In [122]:
process_only_df = process_system_df[PROCESS_ONLY_COLUMNS].copy()
system_only_df = process_system_df[SYSTEM_ONLY_COLUMNS].copy()

# Print description on df

In [123]:
print(f"Full df shape: {process_system_df.shape}")
print(f"System only df shape: {system_only_df.shape}")
print(f"Process only df shape: {process_only_df.shape}")

Full df shape: (8183, 57)
System only df shape: (8183, 14)
Process only df shape: (8183, 11)


# Prepare both datasets (System only and System-Process)

In [124]:
system_only_df = process_df(system_only_df, NON_NEGATIVE_COLS2, OUTLIER_COLS)

In [125]:
system_no_process_df = system_only_df.copy()

In [126]:
system_no_process_df[CPU_SYSTEM_COL] = system_only_df[CPU_SYSTEM_COL] - process_only_df[CPU_PROCESS_COL]
system_no_process_df[MEMORY_SYSTEM_COL] = system_only_df[MEMORY_SYSTEM_COL] - process_only_df[MEMORY_PROCESS_COL]
system_no_process_df[DISK_READ_BYTES_SYSTEM_COL] = system_only_df[DISK_READ_BYTES_SYSTEM_COL] - process_only_df[
    DISK_READ_BYTES_PROCESS_COL]
system_no_process_df[DISK_READ_COUNT_SYSTEM_COL] = system_only_df[DISK_READ_COUNT_SYSTEM_COL] - process_only_df[
    DISK_READ_COUNT_PROCESS_COL]
system_no_process_df[DISK_WRITE_BYTES_SYSTEM_COL] = system_only_df[DISK_WRITE_BYTES_SYSTEM_COL] - process_only_df[
    DISK_WRITE_BYTES_PROCESS_COL]
system_no_process_df[DISK_WRITE_COUNT_SYSTEM_COL] = system_only_df[DISK_WRITE_COUNT_SYSTEM_COL] - process_only_df[
    DISK_WRITE_COUNT_PROCESS_COL]
system_no_process_df[NETWORK_BYTES_SENT_SYSTEM_COL] = system_only_df[NETWORK_BYTES_SENT_SYSTEM_COL] - process_only_df[
    NETWORK_BYTES_SENT_PROCESS_COL]
system_no_process_df[NETWORK_PACKETS_SENT_SYSTEM_COL] = system_only_df[NETWORK_PACKETS_SENT_SYSTEM_COL] - \
                                                        process_only_df[NETWORK_PACKETS_SENT_PROCESS_COL]
system_no_process_df[NETWORK_BYTES_RECEIVED_SYSTEM_COL] = system_only_df[NETWORK_BYTES_RECEIVED_SYSTEM_COL] - \
                                                          process_only_df[NETWORK_BYTES_RECEIVED_PROCESS_COL]
system_no_process_df[NETWORK_PACKETS_RECEIVED_SYSTEM_COL] = system_only_df[NETWORK_PACKETS_RECEIVED_SYSTEM_COL] - \
                                                            process_only_df[NETWORK_PACKETS_RECEIVED_PROCESS_COL]
system_no_process_df[DISK_WRITE_COUNT_SYSTEM_COL] = system_only_df[DISK_WRITE_COUNT_SYSTEM_COL] - process_only_df[
    DISK_WRITE_COUNT_PROCESS_COL]

In [127]:
print(f"System df shape after processing: {system_only_df.shape}")
print(f"System no Process df shape after processing: {system_no_process_df.shape}")

System df shape after processing: (7493, 14)
System no Process df shape after processing: (7493, 14)


# Train System Model

In [128]:
X_system_only = system_only_df.drop(columns=[ENERGY_USAGE_SYSTEM_COL])
y_system_only = system_only_df[ENERGY_USAGE_SYSTEM_COL]

In [129]:
system_scaler, system_model = execute_model(X_system_only, y_system_only)

Actual: 17.29371592
Predicted: 9.054576696244984
Actual: 12.770563733333333
Predicted: 13.60556558648621
Actual: 7.111987259999999
Predicted: 6.865878799307327
Actual: 10.16214345
Predicted: 10.475898707575313
Actual: 58.38825332999999
Predicted: 57.035918158091306
Actual: 7.062218280000001
Predicted: 7.011371686825822
Actual: 7.010462520000001
Predicted: 7.041324166729966
Actual: 46.37035271999999
Predicted: 46.36508891061065
Actual: 17.86560804
Predicted: 14.20855585203064
Actual: 17.410619526666668
Predicted: 13.762881903839979
Actual: 10.91138688
Predicted: 10.421841996118234
Actual: 47.09520888
Predicted: 46.86277199272927
Actual: 6.795401309999999
Predicted: 7.104873534672094
Actual: 59.17796316000001
Predicted: 58.8392105385001
Actual: 11.24647595
Predicted: 14.043388824939273
Actual: 11.231597246666665
Predicted: 13.706619519067184
Actual: 6.66198129
Predicted: 6.8947390197387595
Actual: 7.161712200000001
Predicted: 7.059463578920846
Actual: 10.769249636666666
Predicted: 11.151

# Predict on system-process df

In [130]:
system_no_process_df = system_no_process_df.drop(columns=[ENERGY_USAGE_SYSTEM_COL])

system_no_process_scaled = system_scaler.transform(system_no_process_df)
system_no_process_df_scaled = pd.DataFrame(
    system_no_process_scaled,
    columns=system_no_process_df.columns,
    index=system_no_process_df.index
)

In [131]:
system_no_process_df_scaled.shape

(7493, 13)

In [132]:
energy_predictions_system_without_process = system_model.predict(system_no_process_df_scaled)

# Calculate process energy

In [133]:
energy_predictions_system_without_process

array([10.81245977, 10.24498421,  9.56154198, ...,  7.11307294,
        6.97810831,  7.07971787], shape=(7493,))

In [134]:
system_only_df[ENERGY_USAGE_SYSTEM_COL]

0       10.924691
1       11.044515
2       10.901966
4       10.886528
5       10.960163
          ...    
8100     6.653400
8124     8.041103
8130     7.526159
8179     7.455502
8181     7.495308
Name: system_energy_usage_mwh, Length: 7493, dtype: float64

In [135]:
process_energy_predictions = system_df[ENERGY_USAGE_SYSTEM_COL] - energy_predictions_system_without_process

In [136]:
process_energy_predictions.shape

(7493,)

In [137]:
process_energy_predictions

0       0.112231
1       0.799531
2       1.340424
4       0.910840
5       0.623463
          ...   
8100   -0.217444
8124    0.986267
8130    0.413086
8179    0.477393
8181    0.415590
Name: system_energy_usage_mwh, Length: 7493, dtype: float64

# Update df with process energy

In [138]:
len(process_system_df.index)

8183

In [139]:
process_system_df[ENERGY_USAGE_PROCESS_COL] = process_energy_predictions

In [141]:
process_system_df = process_system_df[process_system_df[ENERGY_USAGE_PROCESS_COL].notna()]

In [142]:
process_system_df

,duration,cpu_usage_seconds_process,memory_mb_relative_process,disk_read_kb_process,disk_read_count_process,disk_write_kb_process,disk_write_count_process,number_of_page_faults_process,network_kb_sent_process,network_packets_sent_process,...,logical_disk_partition_style,logical_disk_number_of_partitions,physical_sector_size,logical_sector_size,bus_type,file_system,design_battery_capacity,fully_charged_battery_capacity,energy_consumption_process_mWh,system_energy_usage_mwh
0,5.260686,0.000000,0.574,0.0,0,0.0,0,147,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,0.112231,10.924691
1,5.318386,0.000000,-0.238,0.0,0,0.0,0,1160,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,0.799531,11.044515
2,5.249743,0.000000,0.258,0.0,0,0.0,0,66,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,1.340424,10.901966
4,5.242309,0.000000,0.199,0.0,0,0.0,0,484,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,0.910840,10.886528
5,5.277767,0.031667,0.133,0.0,0,0.0,0,29388,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,0.623463,10.960163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8100,2.153204,0.882814,0.000,0.0,0,0.0,0,0,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,-0.217444,6.653400
8124,2.299648,0.000000,0.000,0.0,0,0.0,0,0,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,0.986267,8.041103
8130,2.152381,0.000000,0.000,0.0,0,0.0,0,0,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,0.413086,7.526159
8179,2.132174,0.000000,0.000,0.0,0,0.0,0,0,0.0,0,...,GPT,5,512,512,RAID,NTFS,61970,47850,0.477393,7.455502


# Process df - preprocess

In [143]:
FULL_DF_COLUMNS = [
    DURATION_COL, CPU_PROCESS_COL, MEMORY_PROCESS_COL, DISK_READ_BYTES_PROCESS_COL,
    DISK_READ_COUNT_PROCESS_COL, DISK_WRITE_BYTES_PROCESS_COL, DISK_WRITE_COUNT_PROCESS_COL,
    NETWORK_BYTES_RECEIVED_PROCESS_COL, NETWORK_PACKETS_RECEIVED_PROCESS_COL,
    NETWORK_BYTES_SENT_PROCESS_COL, NETWORK_PACKETS_SENT_PROCESS_COL, PAGE_FAULT_PROCESS_COL, CPU_SYSTEM_COL,
    MEMORY_SYSTEM_COL, DISK_READ_BYTES_SYSTEM_COL, DISK_READ_COUNT_SYSTEM_COL, DISK_WRITE_BYTES_SYSTEM_COL,
    DISK_WRITE_COUNT_SYSTEM_COL, DISK_READ_TIME, DISK_WRITE_TIME, NETWORK_BYTES_RECEIVED_SYSTEM_COL,
    NETWORK_PACKETS_RECEIVED_SYSTEM_COL,
    NETWORK_BYTES_SENT_SYSTEM_COL, NETWORK_PACKETS_SENT_SYSTEM_COL, ENERGY_USAGE_PROCESS_COL
]

In [144]:
process_full_df = process_system_df[FULL_DF_COLUMNS]

In [146]:
process_full_df.head()

,duration,cpu_usage_seconds_process,memory_mb_relative_process,disk_read_kb_process,disk_read_count_process,disk_write_kb_process,disk_write_count_process,network_kb_received_process,network_packets_received_process,network_kb_sent_process,...,disk_read_count_system,disk_write_kb_system,disk_write_count_system,disk_read_time_ms_system,disk_write_time_ms_system,network_kb_received_system,network_packets_received_system,network_kb_sent_system,network_packets_sent_system,energy_consumption_process_mWh
0,5.260686,0.000000,0.574,0.0,0,0.0,0,0.0,0,0.0,...,37,6674.5,408,0,0,17,17,0.700,8,0.112231
1,5.318386,0.000000,-0.238,0.0,0,0.0,0,0.0,0,0.0,...,3,1439.5,102,0,0,7,7,3.155,5,0.799531
2,5.249743,0.000000,0.258,0.0,0,0.0,0,0.0,0,0.0,...,9,1062.5,63,0,0,6,6,8.820,108,1.340424
4,5.242309,0.000000,0.199,0.0,0,0.0,0,0.0,0,0.0,...,2,1965.5,37,0,0,35,35,70.788,120,0.910840
5,5.277767,0.031667,0.133,0.0,0,0.0,0,0.0,0,0.0,...,3,1988.0,95,0,0,3,3,0.707,8,0.623463


In [145]:
process_full_df.shape

(7493, 25)

# Train Process Model

In [148]:
process_full_df = process_df(process_full_df, NON_NEGATIVE_COLS2, OUTLIER_COLS)
X_full = process_full_df.drop(columns=[ENERGY_USAGE_PROCESS_COL])
y_full = process_full_df[ENERGY_USAGE_PROCESS_COL]

In [149]:
process_scaler, process_model = execute_model(X_full, y_full)

Actual: 20.446822518221616
Predicted: 19.706956579781323
Actual: -0.8210432132113468
Predicted: -0.04063841121329312
Actual: 11.696102326156584
Predicted: 12.493938385025606
Actual: -1.1633226365513476
Predicted: -0.009975666744029318
Actual: 5.132922011040343
Predicted: 5.485392355192327
Actual: -1.2249542569901557
Predicted: 0.0035874277123550177
Actual: 4.020443511509866
Predicted: -0.2568206250959865
Actual: 1.3636952655011108
Predicted: 0.47010333252041847
Actual: 12.147851069534063
Predicted: 12.533776479426402
Actual: 0.8569164741694983
Predicted: 0.47941641586572886
Actual: -0.20621211903969972
Predicted: -0.12152084052916938
Actual: 0.08734942062138185
Predicted: -0.06846905095092745
Actual: 4.1353307481202535
Predicted: 4.2351956588136375
Actual: -1.9355560187313507
Predicted: -0.36329372001567106
Actual: 0.46634743335850715
Predicted: 0.4650442891190901
Actual: -1.7396200785107165
Predicted: -0.44347266541113645
Actual: 0.15207293677264744
Predicted: -0.4398515788746157
Actu